In [71]:
# prompt: list all files in the following folder  /Users/hemantmangla/Documents/workspace/nandi/scripts/chennai-metro/open-trip-planner/chennai.metro.gtfs, also read the data from those files and put in dataframes

import pandas as pd
import os

# Replace with the actual path to your folder
folder_path = "/Users/hemantmangla/Documents/workspace/nandi/scripts/chennai-subway/open-trip-planner/chennai.suburban.gtfs"

# Check if the folder exists
if not os.path.exists(folder_path):
  print(f"Error: Folder '{folder_path}' not found.")
else:
  dataframes = {}
  for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Assuming GTFS files are text files
      filepath = os.path.join(folder_path, filename)
      try:
        # Attempt to read the file as a CSV (GTFS files are typically CSV-like)
        df = pd.read_csv(filepath)
        dataframes[filename] = df
        print(f"Successfully read '{filename}' into a DataFrame.")
      except pd.errors.ParserError:
        print(f"Warning: Could not parse '{filename}' as a CSV. Skipping.")
      except Exception as e:
        print(f"An error occurred while processing '{filename}': {e}")

  # Now you have a dictionary 'dataframes' where keys are filenames and values are the DataFrames
  # Access them like this:
  # if "agency.txt" in dataframes:
  #     print(dataframes["agency.txt"].head())


Successfully read 'agency.txt' into a DataFrame.
Successfully read 'stop_times.txt' into a DataFrame.
Successfully read 'trips.txt' into a DataFrame.
Successfully read 'stops.txt' into a DataFrame.
Successfully read 'calendar.txt' into a DataFrame.
Successfully read 'routes.txt' into a DataFrame.


In [72]:
routes = dataframes['routes.txt']

stop_times = dataframes['stop_times.txt']

trips = dataframes['trips.txt']


,trip_id,arrival_time,departure_time,stop_sequence,stop_id
0,1,5:50:00,5:50:00,1,MSB
1,1,5:54:00,5:55:00,2,MSF
2,1,5:56:00,5:57:00,3,MPK
3,1,5:59:00,6:00:00,4,MS
4,1,6:02:00,6:03:00,5,MSC
...,...,...,...,...,...
3289,203,24:16:00,24:17:00,14,MSC
3290,203,24:19:00,24:20:00,15,MS
3291,203,24:22:00,24:23:00,16,MPK
3292,203,24:25:00,24:26:00,17,MSF


In [96]:
# prompt: routes has route_id which is inside trips, figure out how many trips each route is having. Along with trips has trip_id which is inside stop_times, figure out how many stops a trip having

import pandas as pd
import numpy as np
# Merge routes and trips dataframes
merged_routes_trips = pd.merge(routes, trips, on='route_id', how='left')

# Group by route_id and count the number of trips for each route
route_trip_counts = merged_routes_trips.groupby('route_id')['trip_id'].count().reset_index()
route_trip_counts['route_long_name'] = np.array(x.replace('Puratchi Thalaivar Dr.M.G.Ramachandran Central', 'MGR').replace('Puratchi Thalaivi Dr.J.Jayalalithaa', '') for x in merged_routes_trips.groupby('route_id')['route_long_name'].first().values)
route_trip_counts.rename(columns={'trip_id': 'trip_count'}, inplace=True)

route_trip_counts.sort_values(by='route_long_name', ascending=False, inplace=True)

# Display the results
print(route_trip_counts)
# # Merge trips and stop_times dataframes
merged_trips_stop_times = pd.merge(trips, stop_times, on='trip_id', how='left')
# # # # Group by trip_id and count the number of stops for each trip
trip_stop_arrays = (
    merged_trips_stop_times
    .sort_values(['trip_id', 'stop_sequence'])  # Ensure ordering
    .groupby('trip_id')['stop_id']
    .agg(list)
    .reset_index()
)
trip_stop_arrays.rename(columns={'stop_id': 'stops'}, inplace=True)

trip_stop_counts = merged_trips_stop_times.groupby('trip_id')['stop_id'].count().reset_index()
trip_stop_counts.rename(columns={'stop_id': 'stop_count'}, inplace=True)

# To figure out trips with same stops count
# print(trip_stop_counts.groupby('stop_count')['trip_id'].count().reset_index())
trip_stop_arrays['stops_string'] = trip_stop_arrays['stops'].apply(lambda stops: '->'.join(map(str, stops)))
print(trip_stop_arrays.groupby('stops_string')['trip_id'].count().reset_index().to_string())
# # Merge the results to have route_id, trip_count, trip_id, and stop_count
# final_df = pd.merge(route_trip_counts, merged_trips_stop_times, on='trip_id', how='left')
# final_df = pd.merge(final_df, trip_stop_counts, on='trip_id', how='left')

# # Display the final dataframe
# print(final_df[['route_id', 'trip_count', 'trip_id', 'stop_count']].head())


       route_id  trip_count    route_long_name
1  86155a4b590f          89  MSB-CGL EMU LOCAL
0  122a6b143830          95  CGL-MSB EMU LOCAL
                                                                            stops_string  trip_id
0  MSB->MSF->MPK->MS->MSC->NBK->MKK->MBM->SP->GDY->STM->PZA->MN->TLM->PV->CMP->TBMS->TBM       89
1  TBM->TBMS->CMP->PV->TLM->MN->PZA->STM->GDY->SP->MBM->MKK->NBK->MSC->MS->MPK->MSF->MSB       94
2                                                                                    nan        1
